In [404]:
from functools import cache
from collections import deque

In [405]:
class Valley(object):
    directions = {
        '<': (0,-1),
        '>': (0,1),
        'v': (1,0),
        '^': (-1,0),
        'o': (0,0),
    }

    directions_inv = {
        v:k for k,v in directions.items()
    }

    def __init__(self,file, start = (-1,0)) -> None:
        self._file = file
        self.load()
        self.entrance = start
        self.elves = self.entrance
        self.exit = (self.size[0],self.size[1]-1)

    @property
    def size(self):
        return (len(self._input) - 2 , len(self._input[0]) -2 )

    def __repr__(self) -> str:
        output = ''
        for r in range(-1,self.size[0]+1):
            for c in range(-1,self.size[1]+1):
                if r < 0 or c < 0:
                    if (r,c) == self.elves:
                        output += 'E'
                    elif (r,c) == self.entrance:
                        output += '.'
                    else:
                        output += '#'
                elif r >= self.size[0] or c >= self.size[1]:
                    if (r,c) == self.elves:
                        output += 'E'
                    elif (r,c) == self.exit:
                        output += '.'
                    else:
                        output += '#'
                else:
                    l = [x for x in self.blizzards if x[0] == (r,c)]
                    if l != []:
                        if len(l) == 1:
                            if l[0][0] == self.elves:
                                output += 'X'
                            else:
                                output += self.directions_inv[l[0][1]]
                        else:
                            if l[0][0] == self.elves:
                                output += 'X'
                            else:
                                output += str(len(l))
                    elif (r,c) == self.elves:
                        output += 'E'
                    else:
                        output += '.'            
            output += '\n'
        return output

    def load(self):
        with open(self._file,'r') as file:
            input = file.read().splitlines()

        blizzards = []
        for r,line in enumerate(input[1:-1]):
            for c,v in enumerate(line[1:-1]):
                if d := self.directions.get(v):
                    blizzards.append(((r,c),d))

        self.blizzards = blizzards
        self._input = input

    @cache
    def get_blizzards(self, time):
        return [(tuple([((x+(y * time)) % z) for x,y,z in zip(blizzard[0],blizzard[1],self.size)])) for blizzard in self.blizzards]

    @cache
    def move_options(self,loc):
        return list(filter(lambda x: (0 <= x[0] < self.size[0] and 0 <= x[1] < self.size[1]) or x == self.exit or x == self.entrance,
        [tuple([x+y for x,y in zip(loc,d)]) for d in self.directions.values()]))

    def solve(self, start = None, exit = None, time = None):

        if start is None:
            start = self.entrance

        if exit is None:
            exit = self.exit

        if time is None:
            time = 0

        q = deque()
        q.append((time,start))
        visited = set()

        tdest = None
        while not tdest:
            t, loc = q.popleft()
            
            if (t,loc) in visited:
                continue
            else:
                visited.add((t,loc))

            next_blizzards = self.get_blizzards(t+1)
            mopts = list(filter(lambda x: x not in next_blizzards,self.move_options(loc)))

            for opt in mopts:
                if opt == exit:
                    tdest = t + 1
                    break
                else:
                    q.append((t+1,opt))


        return tdest - time



In [406]:
a = Valley('./assets/input_day_24.txt')

In [408]:
t1 = a.solve()
print(t1)

t2 = a.solve(a.exit,a.entrance,t1)
t3 = a.solve(time=(t1+t2))

print(t1+ t2 + t3)

257
828
